# Rekomendasi FILM

https://developer.imdb.com/non-commercial-datasets/

# 🎬 Pra-Pemrosesan Dataset IMDb untuk Sistem Rekomendasi Film

Notebook ini bertujuan untuk menyiapkan dataset sistem rekomendasi film berbasis IMDb. Dataset ini dibangun dari beberapa file `.tsv` resmi IMDb, kemudian diproses dan difilter agar hanya mencakup film tahun 2000–2025. File akhir akan disimpan dalam format `.csv` yang siap digunakan dalam pipeline Machine Learning atau aplikasi rekomendasi film.

## 🧭 Alur dan Tahapan Kode

1. **Menentukan Path Dataset dan Output**  
   Mendefinisikan lokasi file input IMDb (`.tsv`) dan path output `.csv` yang akan dihasilkan.

2. **Membaca Data Mentah dari IMDb (.tsv)**  
   File yang dimuat:
   - `title.basics.tsv`: informasi dasar film
   - `title.ratings.tsv`: rating film
   - `title.crew.tsv`: sutradara dan penulis
   - `title.principals.tsv`: aktor utama
   - `name.basics.tsv`: nama asli dari ID orang

3. **Filtering Film Tahun 2000–2025**  
   Hanya memilih baris dengan `titleType = movie` dan tahun antara 2000 hingga 2025.

4. **Menggabungkan Data Film Dasar**  
   Data digabung berdasarkan `tconst` untuk menyatukan informasi dari basics, ratings, dan crew.

5. **Menggabungkan Data Aktor Utama**  
   Mengelompokkan aktor/aktris berdasarkan ID film (`tconst`), lalu menggabungkannya ke dataset utama.

6. **Membuat Dictionary ID → Nama Asli**  
   Menghubungkan ID dari aktor, penulis, dan sutradara ke nama asli mereka berdasarkan file `name.basics.tsv`.

7. **Konversi ID menjadi Nama**  
   Kolom `directors`, `writers`, dan `actors` dikonversi dari ID menjadi string nama menggunakan dictionary.

8. **Memilih Kolom Penting**  
   Hanya memilih kolom relevan yang dibutuhkan untuk sistem rekomendasi, seperti judul, tahun, genre, rating, dan nama-nama kru film.

9. **Menyimpan Hasil ke Format CSV**  
   Dataset final disimpan sebagai file `.csv` bernama `imdb_recommendation_ready.csv` di folder output yang telah ditentukan.

In [ ]:
import os
import pandas as pd

# 📌 1. Path dataset
DATASET_PATH = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/FILM/Dataset"  # Ubah sesuai path lokalmu
DATASET_out="/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2"
OUTPUT_CSV = os.path.join(DATASET_out, "imdb_recommendation_ready.csv")

# 📌 2. Load file .tsv utama
print("📥 Membaca file .tsv...")
df_basics = pd.read_csv(os.path.join(DATASET_PATH, "title.basics.tsv"), sep="\t", dtype=str, na_values="\\N")
df_ratings = pd.read_csv(os.path.join(DATASET_PATH, "title.ratings.tsv"), sep="\t", dtype=str, na_values="\\N")
df_crew = pd.read_csv(os.path.join(DATASET_PATH, "title.crew.tsv"), sep="\t", dtype=str, na_values="\\N")
df_principals = pd.read_csv(os.path.join(DATASET_PATH, "title.principals.tsv"), sep="\t", dtype=str, na_values="\\N")
df_names = pd.read_csv(os.path.join(DATASET_PATH, "name.basics.tsv"), sep="\t", dtype=str, na_values="\\N")

# 📌 3. Filter: hanya movie dan tahun 2000–2025
print("🔎 Filter hanya film bioskop tahun 2000–2025...")
df_basics = df_basics[df_basics["titleType"] == "movie"]
df_basics["startYear"] = pd.to_numeric(df_basics["startYear"], errors="coerce")
df_basics = df_basics[df_basics["startYear"].between(2000, 2025)]

# 📌 4. Gabungkan basics + ratings + crew
print("🔄 Menggabungkan informasi film...")
df = df_basics.set_index("tconst").join(df_ratings.set_index("tconst")).reset_index()
df = df.set_index("tconst").join(df_crew.set_index("tconst")).reset_index()

# 📌 5. Gabungkan aktor utama
print("🎭 Menggabungkan aktor utama...")
df_actors = df_principals[df_principals["category"].isin(["actor", "actress"])]
df_actors_grouped = df_actors.groupby("tconst")["nconst"].apply(lambda x: ", ".join(x.dropna())).reset_index()
df_actors_grouped.rename(columns={"nconst": "actors"}, inplace=True)
df = df.set_index("tconst").join(df_actors_grouped.set_index("tconst")).reset_index()

# 📌 6. Buat dictionary ID → nama asli
print("👥 Membuat kamus ID ke nama...")
id_to_name = df_names.set_index("nconst")["primaryName"].to_dict()

# 📌 7. Fungsi konversi ID → nama
def convert_ids_to_names(id_list):
    try:
        if pd.isna(id_list) or id_list is None:
            return "Tidak tersedia"
        id_list = str(id_list).strip()
        if not id_list or id_list.lower() in ["nan", "none", "null"]:
            return "Tidak tersedia"
        return ", ".join([id_to_name.get(n.strip(), "Tidak tersedia") for n in id_list.split(",")])
    except Exception as e:
        return "Tidak tersedia"

# 📌 8. Konversi ID → nama di kolom: directors, writers, actors
print("🔄 Mengonversi ID menjadi nama...")
df["directors"] = df["directors"].fillna("").astype(str).map(convert_ids_to_names)
df["writers"] = df["writers"].fillna("").astype(str).map(convert_ids_to_names)
df["actors"] = df["actors"].fillna("").astype(str).map(convert_ids_to_names)

# 📌 9. Pilih kolom penting
print("📋 Memilih kolom final...")
columns_needed = [
    "tconst", "primaryTitle", "originalTitle", "startYear", "runtimeMinutes",
    "genres", "averageRating", "numVotes",
    "directors", "writers", "actors"
]
df_final = df[columns_needed]

# 📌 10. Simpan ke CSV
print(f"💾 Menyimpan ke `{OUTPUT_CSV}`...")
df_final.to_csv(OUTPUT_CSV, index=False)
print("✅ File siap digunakan untuk sistem rekomendasi.")

📥 Membaca file .tsv...
🔎 Filter hanya film bioskop tahun 2000–2025...
🔄 Menggabungkan informasi film...
🎭 Menggabungkan aktor utama...
👥 Membuat kamus ID ke nama...
🔄 Mengonversi ID menjadi nama...
📋 Memilih kolom final...
💾 Menyimpan ke `/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb_recommendation_ready.csv`...
✅ File siap digunakan untuk sistem rekomendasi.


# 📂 Eksplorasi Awal Dataset Film IMDb

Notebook ini melanjutkan dari tahap sebelumnya, di mana kita telah menyimpan file `imdb_recommendation_ready.csv` hasil pemrosesan. Tahap ini fokus pada eksplorasi awal terhadap struktur dan isi dataset.

## 🧭 Alur dan Tahapan Kode

1. **Menentukan Path Dataset**
   - Path `file_path` disesuaikan dengan lokasi penyimpanan file CSV hasil preprocessing sebelumnya.

2. **Memuat Dataset**
   - Dataset dibaca menggunakan `pandas.read_csv()` dan disimpan ke variabel `df`.

3. **Menampilkan Informasi Umum Dataset**
   - Menampilkan jumlah baris dan kolom untuk mengetahui ukuran dataset.
   - Menampilkan semua nama kolom yang tersedia agar pengguna memahami struktur data.

4. **Melihat Contoh Data**
   - Menampilkan 5 baris pertama menggunakan `.head(5)` dalam format tabel Markdown.

5. **Analisis Statistik Sederhana**
   - Mengonversi kolom `averageRating` dan `startYear` ke numerik (jika belum).
   - Menampilkan statistik deskriptif seperti mean, min, max, std untuk kolom rating dan tahun rilis menggunakan `.describe()`.

> Hasil dari tahap ini akan membantu kita memahami distribusi data dan memvalidasi bahwa data sudah sesuai sebelum masuk ke tahap visualisasi atau modeling.

In [ ]:
file_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb_recommendation_ready.csv"  # Ubah jika lokasimu berbeda


import pandas as pd

# Path ke file hasil


# Load file CSV
df = pd.read_csv(file_path)

# 🔍 Informasi umum
print("🔍 Ukuran dataset:")
print(f"- Jumlah Baris: {df.shape[0]}")
print(f"- Jumlah Kolom: {df.shape[1]}")

print("\n🧾 Kolom yang tersedia:")
for col in df.columns:
    print(f"- {col}")

# 📄 Tampilkan 5 baris pertama
print("\n📄 Contoh isi data:")
print(df.head(5).to_markdown())

# 📊 Statistik rating dan tahun
df["averageRating"] = pd.to_numeric(df["averageRating"], errors="coerce")
df["startYear"] = pd.to_numeric(df["startYear"], errors="coerce")
print("\n📊 Statistik Rating dan Tahun:")
print(df[["averageRating", "startYear"]].describe().to_markdown())

🔍 Ukuran dataset:
- Jumlah Baris: 347398
- Jumlah Kolom: 11

🧾 Kolom yang tersedia:
- tconst
- primaryTitle
- originalTitle
- startYear
- runtimeMinutes
- genres
- averageRating
- numVotes
- directors
- writers
- actors

📄 Contoh isi data:
|    | tconst    | primaryTitle                                       | originalTitle                             |   startYear |   runtimeMinutes | genres                 |   averageRating |   numVotes | directors                    | writers                                     | actors                                                                                                                                                                        |
|---:|:----------|:---------------------------------------------------|:------------------------------------------|------------:|-----------------:|:-----------------------|----------------:|-----------:|:-----------------------------|:--------------------------------------------|:--------------------

# 🎬 Enrichment Dataset IMDb dengan Informasi dari TMDb

Pada tahap ini, kita akan memperkaya dataset IMDb yang telah diproses sebelumnya dengan data tambahan dari TMDb (poster dan sinopsis/overview). Proses dilakukan secara batch dan paralel (multithreading) untuk mempercepat pengambilan data.

---

## ⚙️ 1. Konfigurasi dan Inisialisasi

- **TMDB_API_KEY**: Kunci akses API TMDb.
- **Path input/output**: Mendefinisikan lokasi file CSV input (`imdb_recommendation_ready.csv`) dan file output hasil enrichment (`imdb_tmdb_enriched.csv`).
- **BATCH_SIZE**: Jumlah maksimum film yang diproses dalam satu batch (default: 10.000).
- **SAVE_INTERVAL**: Menentukan interval simpan otomatis per beberapa film yang berhasil diproses.
- **MAX_WORKERS**: Jumlah thread paralel yang digunakan untuk mempercepat pengambilan data API (default: 10).

---

## 📥 2. Load Dataset

- Jika file hasil enrichment sudah ada (`output_csv`), proses dilanjutkan dari file tersebut.
- Jika belum ada, maka:
  - File input akan dimuat.
  - Dua kolom tambahan (`poster_url`, `overview`) dibuat jika belum ada.
  - File disimpan sebagai file output awal.

---

## 🌐 3. Fungsi Ambil Data dari TMDb (`fetch_tmdb_data`)

- Menerima `imdb_id`.
- Mengakses endpoint TMDb `find/<imdb_id>`.
- Jika ditemukan:
  - Ambil **poster_path** dan **overview**.
  - Kembalikan URL poster dan overview.
- Jika gagal, kembalikan nilai `"NOT_FOUND"`.

---

## 📋 4. Filter Film yang Belum Diperkaya

- Hanya memilih baris dengan `poster_url` dan `overview` masih kosong (`NaN`) dan belum diberi label `"NOT_FOUND"`.
- Maksimum jumlah yang diambil sesuai `BATCH_SIZE`.

---

## ⚡️ 5. Proses Paralel (Multithreading)

- Menggunakan `ThreadPoolExecutor` untuk menjalankan permintaan API secara paralel.
- Untuk setiap hasil:
  - Cek apakah data masih kosong sebelum mengupdate.
  - Update kolom `poster_url` dan `overview` secara langsung pada dataframe.
  - Simpan data secara berkala tiap `SAVE_INTERVAL` entri atau jika sudah selesai semua.

---

## ✅ 6. Output

- Setelah seluruh batch diproses, hasil disimpan kembali ke file CSV.
- Informasi status ditampilkan untuk setiap film yang berhasil diproses.

---

> Proses ini sangat penting untuk mendapatkan visual dan sinopsis yang berguna saat menampilkan rekomendasi kepada pengguna dalam sistem rekomendasi film.

In [ ]:
import pandas as pd
import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# === Konfigurasi ===
TMDB_API_KEY = "1ec75235bb4ad6c9a7d6b6b8eac6d44e"
input_csv = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb_recommendation_ready.csv"
output_csv = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_enriched.csv"
image_base = "https://image.tmdb.org/t/p/w500"
BATCH_SIZE = 10000
SAVE_INTERVAL = 5000
MAX_WORKERS = 10

# === Load Data ===
if os.path.exists(output_csv):
    df = pd.read_csv(output_csv)
    print(f"🔄 Melanjutkan dari file: {output_csv}")
else:
    df = pd.read_csv(input_csv)
    for col in ["poster_url", "overview"]:
        if col not in df.columns:
            df[col] = None
    df.to_csv(output_csv, index=False)
    print(f"🆕 File baru dibuat: {output_csv}")

# === Fungsi Ambil Data dari TMDb ===
def fetch_tmdb_data(imdb_id):
    url = f"https://api.themoviedb.org/3/find/{imdb_id}?api_key={TMDB_API_KEY}&external_source=imdb_id"
    try:
        res = requests.get(url)
        if res.status_code == 200:
            data = res.json()
            if data.get("movie_results"):
                movie = data["movie_results"][0]
                poster = movie.get("poster_path")
                overview = movie.get("overview", "")
                poster_url = f"{image_base}{poster}" if poster else "NOT_FOUND"
                overview = overview if overview else "NOT_FOUND"
                return imdb_id, poster_url, overview
    except Exception as e:
        print(f"⚠️ Error untuk {imdb_id}: {e}")
    return imdb_id, "NOT_FOUND", "NOT_FOUND"

# === Filter Film yang Belum Diproses (masih NaN dan bukan NOT_FOUND) ===
pending = df[
    ((df["poster_url"].isna()) | (df["overview"].isna())) &
    ((df["poster_url"] != "NOT_FOUND") & (df["overview"] != "NOT_FOUND"))
]
print(f"🚀 Siap memproses {len(pending)} film...")
pending_batch = pending.head(BATCH_SIZE)

# === Mulai Proses Multithread ===
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(fetch_tmdb_data, row["tconst"]): row for _, row in pending_batch.iterrows()}
    results = []
    for i, future in enumerate(as_completed(futures)):
        imdb_id, poster_url, overview = future.result()

        # Update hanya jika masih kosong
        if df.loc[df["tconst"] == imdb_id, "poster_url"].isna().any():
            df.loc[df["tconst"] == imdb_id, "poster_url"] = poster_url
        if df.loc[df["tconst"] == imdb_id, "overview"].isna().any():
            df.loc[df["tconst"] == imdb_id, "overview"] = overview

        print(f"✅ [{i+1}/{len(pending_batch)}] {imdb_id} selesai")

        # Simpan setiap kelipatan SAVE_INTERVAL
        if (i + 1) % SAVE_INTERVAL == 0 or (i + 1) == len(pending_batch):
            df.to_csv(output_csv, index=False)
            print(f"💾 Data disimpan setelah {i + 1} film diproses.")

print(f"\n✅ Batch {BATCH_SIZE} selesai. Data akhir disimpan ke {output_csv}")

<ipython-input-1-3853ee4a2ee7>:17: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output_csv)


Streaming output truncated to the last 5000 lines.
✅ [5004/10000] tt32080121 selesai
✅ [5005/10000] tt3208026 selesai
✅ [5006/10000] tt32080770 selesai
✅ [5007/10000] tt32080876 selesai
✅ [5008/10000] tt3208116 selesai
✅ [5009/10000] tt32080311 selesai
✅ [5010/10000] tt32081990 selesai
✅ [5011/10000] tt32080527 selesai
✅ [5012/10000] tt3208058 selesai
✅ [5013/10000] tt3208150 selesai
✅ [5014/10000] tt32082847 selesai
✅ [5015/10000] tt32082861 selesai
✅ [5016/10000] tt3208228 selesai
✅ [5017/10000] tt32082245 selesai
✅ [5018/10000] tt32083051 selesai
✅ [5019/10000] tt3208258 selesai
✅ [5020/10000] tt32083102 selesai
✅ [5021/10000] tt32083134 selesai
✅ [5022/10000] tt32082536 selesai
✅ [5023/10000] tt32082258 selesai
✅ [5024/10000] tt32083336 selesai
✅ [5025/10000] tt32083311 selesai
✅ [5026/10000] tt3208272 selesai
✅ [5027/10000] tt32083116 selesai
✅ [5028/10000] tt3208338 selesai
✅ [5029/10000] tt32083292 selesai
✅ [5030/10000] tt32083293 selesai
✅ [5031/10000] tt32083606 selesai
✅ [50

# 📊 Evaluasi Kelengkapan Dataset Setelah Enrichment dari TMDb

Setelah proses pengambilan data poster dan sinopsis dari TMDb, langkah ini digunakan untuk menganalisis **kualitas dan kelengkapan** dataset hasil enrichment. Analisis ini berguna untuk:

- Mengetahui berapa persen data yang sudah diproses.
- Mengidentifikasi entri yang belum memiliki informasi penting.
- Menentukan apakah perlu pengambilan ulang dari API atau membuang entri tidak lengkap.

---

## 📥 1. Load Dataset Hasil Enrichment

- Dataset `imdb_tmdb_enriched.csv` dimuat menggunakan `pandas`.
- Total jumlah baris dihitung.
- Data akan dianalisis berdasarkan status kelengkapan kolom `poster_url` dan `overview`.

---

## 🧮 2. Identifikasi Baris Belum Terisi

- Baris yang **belum diproses sama sekali** adalah baris dengan `poster_url` dan `overview` bernilai `NaN`.
- Baris lain, termasuk yang bernilai `"NOT_FOUND"`, dianggap **sudah diproses**.

**Hasil Ringkasan Ditampilkan:**
- Jumlah baris total.
- Jumlah baris sudah diproses (valid atau `NOT_FOUND`).
- Jumlah baris belum pernah diproses.

---

## 🧾 3. Sampel Isi Dataset

- Menampilkan **20 baris pertama** dari kolom penting seperti:
  - `primaryTitle`
  - `startYear`
  - `poster_url`
  - `overview`
  - `overview_id` (jika ada)

---

## 🔍 4. Rekap Data Tidak Lengkap

Dilakukan pemeriksaan lebih lanjut untuk menghitung jumlah:

- Baris tanpa nilai `averageRating`.
- Baris tanpa genre (`genres` kosong).
- Baris dengan nilai `"NOT_FOUND"` di `overview` dan `poster_url`.
- Baris tanpa `startYear`.
- Baris yang **belum pernah diambil datanya** dari API TMDb (masih `NaN`, bukan `"NOT_FOUND"`).

---

## 📌 Tujuan Akhir

Evaluasi ini bertujuan untuk:

- Menentukan **data yang siap digunakan** dalam sistem rekomendasi.
- Mengidentifikasi **baris yang harus diambil ulang** atau dibuang.
- Memberikan ringkasan visual kepada tim atau dosen sebagai bukti kelengkapan data.

In [ ]:
import pandas as pd

# Path ke file enriched
file_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_enriched.csv"

# Load data
df = pd.read_csv(file_path)
total_rows = len(df)

# Baris yang belum terisi sama sekali (poster dan overview keduanya NaN)
unfilled_rows = df[df["poster_url"].isna() & df["overview"].isna()]
count_unfilled = len(unfilled_rows)
percent_unfilled = (count_unfilled / total_rows) * 100

# Sisanya dianggap terisi (sudah diproses), baik valid atau "NOT_FOUND"
count_filled = total_rows - count_unfilled
percent_filled = (count_filled / total_rows) * 100

# Tampilkan ringkasan
print("📊 Ringkasan Status Dataset (NOT_FOUND dianggap terisi):\n")
print(f"- Jumlah total baris        : {total_rows}")
print(f"- Jumlah baris sudah diproses : {count_filled} ({percent_filled:.2f}%) — memiliki minimal poster/sinopsis atau NOT_FOUND")
print(f"- Jumlah baris belum terisi  : {count_unfilled} ({percent_unfilled:.2f}%) — masih kosong (belum pernah diproses)")

# 📄 Tampilkan 20 baris pertama sebagai sampel
columns_to_show = ["primaryTitle", "startYear", "poster_url", "overview", "overview_id"]
columns_exist = [col for col in columns_to_show if col in df.columns]

print(f"\n📄 Menampilkan isi 20 baris pertama untuk kolom: {columns_exist}")
print(df[columns_exist].head(5).to_markdown())


# Hitung kondisi yang diminta
rating_missing = df["averageRating"].isna().sum()
genre_missing = df["genres"].isna().sum()
overview_nf = (df["overview"] == "NOT_FOUND").sum()
poster_nf = (df["poster_url"] == "NOT_FOUND").sum()
year_missing = df["startYear"].isna().sum()

# Tambahan: Baris yang BELUM dilakukan pengambilan data TMDb (masih kosong dan bukan "NOT_FOUND")
pending_tmdb = df[
    ((df["poster_url"].isna()) & (df["poster_url"] != "NOT_FOUND")) |
    ((df["overview"].isna()) & (df["overview"] != "NOT_FOUND"))
]
pending_tmdb_count = len(pending_tmdb)

# Tampilkan hasil
print("📊 Rekap Data Tidak Lengkap / Potensial Dibuang:")
print(f"- Baris tanpa rating              : {rating_missing}")
print(f"- Baris tanpa genre               : {genre_missing}")
print(f"- Baris overview 'NOT_FOUND'      : {overview_nf}")
print(f"- Baris poster 'NOT_FOUND'        : {poster_nf}")
print(f"- Baris tanpa tahun rilis         : {year_missing}")
print(f"- Baris BELUM diambil dari API TMDb: {pending_tmdb_count}")

<ipython-input-23-c08aece0ddd7>:7: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


📊 Ringkasan Status Dataset (NOT_FOUND dianggap terisi):

- Jumlah total baris        : 347398
- Jumlah baris sudah diproses : 231450 (66.62%) — memiliki minimal poster/sinopsis atau NOT_FOUND
- Jumlah baris belum terisi  : 115948 (33.38%) — masih kosong (belum pernah diproses)

📄 Menampilkan isi 20 baris pertama untuk kolom: ['primaryTitle', 'startYear', 'poster_url', 'overview']
|    | primaryTitle                                       |   startYear | poster_url                                                      | overview                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

# 🧹 Pembersihan Dataset IMDb yang Telah Diperkaya (Cleaned Version)

Setelah proses pengambilan data dari TMDb (poster dan overview), tidak semua baris berhasil mendapatkan data lengkap. Oleh karena itu, dilakukan proses **pembersihan** untuk menyaring entri yang memiliki informasi penting agar siap digunakan dalam sistem rekomendasi.

---

## 📥 1. Muat Dataset Hasil Enrichment

- Dataset `imdb_tmdb_enriched.csv` dimuat menggunakan `pandas`.
- Lokasi file disesuaikan dengan Google Drive.
- Jumlah total baris sebelum dibersihkan dicetak.

---

## 🧼 2. Kriteria Pembersihan Data

Dataset hanya akan **dipertahankan** jika memenuhi semua syarat berikut:

- **Memiliki rating** (`averageRating` tidak kosong).
- **Memiliki genre** (`genres` tidak kosong).
- **Tidak memiliki overview "NOT_FOUND"**:
  - Nilai `NaN` (belum diproses) tetap dipertahankan.
  - Hanya baris dengan overview eksplisit `"NOT_FOUND"` yang dibuang.

---

## 💾 3. Simpan Dataset Bersih

- Dataset yang sudah disaring disimpan dalam file baru `imdb_tmdb_cleaned.csv`.
- Jumlah baris setelah pembersihan ditampilkan sebagai perbandingan.

---

## 🎯 Tujuan Akhir

- Menghasilkan dataset yang **lebih layak digunakan** dalam training atau evaluasi sistem rekomendasi film.
- Mengurangi entri yang **tidak bermakna atau gagal diperkaya** dari TMDb.
- Siap digunakan dalam tahap eksplorasi lanjutan atau pembuatan embedding teks.

In [ ]:
import pandas as pd

# Path ke file enriched
file_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_enriched.csv"
output_cleaned = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_cleaned.csv"

# Load data
df = pd.read_csv(file_path)

# Tampilkan ukuran awal
print(f"📦 Dataset awal: {len(df)} baris")

# Tetapkan baris yang:
# - memiliki rating
# - memiliki genre
# - overview TIDAK berisi "NOT_FOUND"
# - biarkan overview kosong (NaN) jika belum diproses
df_clean = df[
    df["averageRating"].notna() &                # Rating tidak kosong
    df["genres"].notna() &                       # Genre tidak kosong
    ((df["overview"].isna()) | (df["overview"] != "NOT_FOUND"))  # Buang yang overview == "NOT_FOUND" saja
]

# Simpan hasil
df_clean.to_csv(output_cleaned, index=False)
print(f"✅ Dataset bersih disimpan: {output_cleaned}")
print(f"📉 Jumlah baris setelah dibersihkan: {len(df_clean)}")

<ipython-input-16-5b6cce962d50>:8: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


📦 Dataset awal: 347398 baris
✅ Dataset bersih disimpan: /content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_cleaned.csv
📉 Jumlah baris setelah dibersihkan: 160994


In [ ]:
import pandas as pd

# Path ke file enriched
file_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_cleaned.csv"

# Load data
df = pd.read_csv(file_path)

# Hitung kondisi yang diminta
rating_missing = df["averageRating"].isna().sum()
genre_missing = df["genres"].isna().sum()
overview_nf = (df["overview"] == "NOT_FOUND").sum()
poster_nf = (df["poster_url"] == "NOT_FOUND").sum()
year_missing = df["startYear"].isna().sum()

# Tambahan: Baris yang BELUM dilakukan pengambilan data TMDb (masih kosong dan bukan "NOT_FOUND")
pending_tmdb = df[
    ((df["poster_url"].isna()) & (df["poster_url"] != "NOT_FOUND")) |
    ((df["overview"].isna()) & (df["overview"] != "NOT_FOUND"))
]
pending_tmdb_count = len(pending_tmdb)

# Tampilkan hasil
print("📊 Rekap Data Tidak Lengkap / Potensial Dibuang:")
print(f"- Baris tanpa rating              : {rating_missing}")
print(f"- Baris tanpa genre               : {genre_missing}")
print(f"- Baris overview 'NOT_FOUND'      : {overview_nf}")
print(f"- Baris poster 'NOT_FOUND'        : {poster_nf}")
print(f"- Baris tanpa tahun rilis         : {year_missing}")
print(f"- Baris BELUM diambil dari API TMDb: {pending_tmdb_count}")


# Baris yang belum terisi sama sekali (poster dan overview keduanya NaN)
unfilled_rows = df[df["poster_url"].isna() & df["overview"].isna()]
count_unfilled = len(unfilled_rows)
percent_unfilled = (count_unfilled / total_rows) * 100

# Sisanya dianggap terisi (sudah diproses), baik valid atau "NOT_FOUND"
count_filled = total_rows - count_unfilled
percent_filled = (count_filled / total_rows) * 100

# Tampilkan ringkasan
print("📊 Ringkasan Status Dataset (NOT_FOUND dianggap terisi):\n")
print(f"- Jumlah total baris        : {total_rows}")
print(f"- Jumlah baris sudah diproses : {count_filled} ({percent_filled:.2f}%) — memiliki minimal poster/sinopsis atau NOT_FOUND")
print(f"- Jumlah baris belum terisi  : {count_unfilled} ({percent_unfilled:.2f}%) — masih kosong (belum pernah diproses)")

# 📄 Tampilkan 20 baris pertama sebagai sampel
columns_to_show = ["primaryTitle", "startYear", "poster_url", "overview", "overview_id"]
columns_exist = [col for col in columns_to_show if col in df.columns]

print(f"\n📄 Menampilkan isi 20 baris pertama untuk kolom: {columns_exist}")
print(df[columns_exist].head(5).to_markdown())

<ipython-input-24-07a30404e085>:7: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


📊 Rekap Data Tidak Lengkap / Potensial Dibuang:
- Baris tanpa rating              : 0
- Baris tanpa genre               : 0
- Baris overview 'NOT_FOUND'      : 0
- Baris poster 'NOT_FOUND'        : 8183
- Baris tanpa tahun rilis         : 0
- Baris BELUM diambil dari API TMDb: 59849
📊 Ringkasan Status Dataset (NOT_FOUND dianggap terisi):

- Jumlah total baris        : 347398
- Jumlah baris sudah diproses : 287549 (82.77%) — memiliki minimal poster/sinopsis atau NOT_FOUND
- Jumlah baris belum terisi  : 59849 (17.23%) — masih kosong (belum pernah diproses)

📄 Menampilkan isi 20 baris pertama untuk kolom: ['primaryTitle', 'startYear', 'poster_url', 'overview']
|    | primaryTitle                                       |   startYear | poster_url                                                      | overview                                                                                                                                                                                          

In [ ]:
import pandas as pd
import os
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed

# === Konfigurasi ===
TMDB_API_KEY = "1ec75235bb4ad6c9a7d6b6b8eac6d44e"
input_csv = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_cleaned.csv"
output_csv = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Filal_clean.csv"
image_base = "https://image.tmdb.org/t/p/w500"
BATCH_SIZE = 39849
SAVE_INTERVAL = 5000
MAX_WORKERS = 10

# === Load Data ===
if os.path.exists(output_csv):
    df = pd.read_csv(output_csv)
    print(f"🔄 Melanjutkan dari file: {output_csv}")
else:
    df = pd.read_csv(input_csv)
    for col in ["poster_url", "overview"]:
        if col not in df.columns:
            df[col] = None
    df.to_csv(output_csv, index=False)
    print(f"🆕 File baru dibuat: {output_csv}")

# === Fungsi Ambil Data dari TMDb ===
def fetch_tmdb_data(imdb_id):
    url = f"https://api.themoviedb.org/3/find/{imdb_id}?api_key={TMDB_API_KEY}&external_source=imdb_id"
    try:
        res = requests.get(url)
        if res.status_code == 200:
            data = res.json()
            if data.get("movie_results"):
                movie = data["movie_results"][0]
                poster = movie.get("poster_path")
                overview = movie.get("overview", "")
                poster_url = f"{image_base}{poster}" if poster else "NOT_FOUND"
                overview = overview if overview else "NOT_FOUND"
                return imdb_id, poster_url, overview
    except Exception as e:
        print(f"⚠️ Error untuk {imdb_id}: {e}")
    return imdb_id, "NOT_FOUND", "NOT_FOUND"

# === Filter Film yang Belum Diproses (masih NaN dan bukan NOT_FOUND) ===
pending = df[
    ((df["poster_url"].isna()) | (df["overview"].isna())) &
    ((df["poster_url"] != "NOT_FOUND") & (df["overview"] != "NOT_FOUND"))
]
print(f"🚀 Siap memproses {len(pending)} film...")
pending_batch = pending.head(BATCH_SIZE)

# === Mulai Proses Multithread ===
with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
    futures = {executor.submit(fetch_tmdb_data, row["tconst"]): row for _, row in pending_batch.iterrows()}
    results = []
    for i, future in enumerate(as_completed(futures)):
        imdb_id, poster_url, overview = future.result()

        # Update hanya jika masih kosong
        if df.loc[df["tconst"] == imdb_id, "poster_url"].isna().any():
            df.loc[df["tconst"] == imdb_id, "poster_url"] = poster_url
        if df.loc[df["tconst"] == imdb_id, "overview"].isna().any():
            df.loc[df["tconst"] == imdb_id, "overview"] = overview

        print(f"✅ [{i+1}/{len(pending_batch)}] {imdb_id} selesai")

        # Simpan setiap kelipatan SAVE_INTERVAL
        if (i + 1) % SAVE_INTERVAL == 0 or (i + 1) == len(pending_batch):
            df.to_csv(output_csv, index=False)
            print(f"💾 Data disimpan setelah {i + 1} film diproses.")

print(f"\n✅ Batch {BATCH_SIZE} selesai. Data akhir disimpan ke {output_csv}")

<ipython-input-26-4de0c3bbfe86>:17: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(output_csv)


Streaming output truncated to the last 5000 lines.
✅ [34854/39849] tt9019312 selesai
✅ [34855/39849] tt9020400 selesai
✅ [34856/39849] tt9020334 selesai
✅ [34857/39849] tt9020586 selesai
✅ [34858/39849] tt9020558 selesai
✅ [34859/39849] tt9020638 selesai
✅ [34860/39849] tt9020648 selesai
✅ [34861/39849] tt9020602 selesai
✅ [34862/39849] tt9020786 selesai
✅ [34863/39849] tt9020644 selesai
✅ [34864/39849] tt9020894 selesai
✅ [34865/39849] tt9021174 selesai
✅ [34866/39849] tt9021140 selesai
✅ [34867/39849] tt9021032 selesai
✅ [34868/39849] tt9020768 selesai
✅ [34869/39849] tt9021066 selesai
✅ [34870/39849] tt9021092 selesai
✅ [34871/39849] tt9021242 selesai
✅ [34872/39849] tt9021194 selesai
✅ [34873/39849] tt9021374 selesai
✅ [34874/39849] tt9021184 selesai
✅ [34875/39849] tt9021234 selesai
✅ [34876/39849] tt9021998 selesai
✅ [34877/39849] tt9021260 selesai
✅ [34878/39849] tt9023136 selesai
✅ [34879/39849] tt9023752 selesai
✅ [34880/39849] tt9021508 selesai
✅ [34881/39849] tt9023894 seles

In [ ]:
import pandas as pd

# Path ke file enriched
file_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Filal_clean.csv"

# Load data
df = pd.read_csv(file_path)
total_rows = len(df)

# Baris yang belum terisi sama sekali (poster dan overview keduanya NaN)
unfilled_rows = df[df["poster_url"].isna() & df["overview"].isna()]
count_unfilled = len(unfilled_rows)
percent_unfilled = (count_unfilled / total_rows) * 100

# Sisanya dianggap terisi (sudah diproses), baik valid atau "NOT_FOUND"
count_filled = total_rows - count_unfilled
percent_filled = (count_filled / total_rows) * 100

# Tampilkan ringkasan
print("📊 Ringkasan Status Dataset (NOT_FOUND dianggap terisi):\n")
print(f"- Jumlah total baris        : {total_rows}")
print(f"- Jumlah baris sudah diproses : {count_filled} ({percent_filled:.2f}%) — memiliki minimal poster/sinopsis")
print(f"- Jumlah baris belum terisi  : {count_unfilled} ({percent_unfilled:.2f}%) — masih kosong (belum pernah diproses)")


# Hitung kondisi yang diminta
rating_missing = df["averageRating"].isna().sum()
genre_missing = df["genres"].isna().sum()
overview_nf = (df["overview"] == "NOT_FOUND").sum()
poster_nf = (df["poster_url"] == "NOT_FOUND").sum()
year_missing = df["startYear"].isna().sum()

# Tambahan: Baris yang BELUM dilakukan pengambilan data TMDb (masih kosong dan bukan "NOT_FOUND")
pending_tmdb = df[
    ((df["poster_url"].isna()) & (df["poster_url"] != "NOT_FOUND")) |
    ((df["overview"].isna()) & (df["overview"] != "NOT_FOUND"))
]
pending_tmdb_count = len(pending_tmdb)

# Tampilkan hasil
print("📊 Rekap Data Tidak Lengkap / Potensial Dibuang:")
print(f"- Baris tanpa rating              : {rating_missing}")
print(f"- Baris tanpa genre               : {genre_missing}")
print(f"- Baris overview 'NOT_FOUND'      : {overview_nf}")
print(f"- Baris poster 'NOT_FOUND'        : {poster_nf}")
print(f"- Baris tanpa tahun rilis         : {year_missing}")
print(f"- Baris BELUM diambil dari API TMDb: {pending_tmdb_count}")

📊 Ringkasan Status Dataset (NOT_FOUND dianggap terisi):

- Jumlah total baris        : 160994
- Jumlah baris sudah diproses : 160994 (100.00%) — memiliki minimal poster/sinopsis
- Jumlah baris belum terisi  : 0 (0.00%) — masih kosong (belum pernah diproses)
📊 Rekap Data Tidak Lengkap / Potensial Dibuang:
- Baris tanpa rating              : 0
- Baris tanpa genre               : 0
- Baris overview 'NOT_FOUND'      : 12380
- Baris poster 'NOT_FOUND'        : 20672
- Baris tanpa tahun rilis         : 0
- Baris BELUM diambil dari API TMDb: 0


# 🧼 Finalisasi Dataset IMDb yang Sempurna

Tahapan ini merupakan **penyaringan akhir (final clean)** untuk menghasilkan dataset IMDb yang benar-benar siap digunakan dalam sistem rekomendasi film berbasis teks.

---

## 📥 1. Memuat Dataset Hasil Pembersihan Awal

- File sumber: `imdb_tmdb_Filal_clean.csv`
- Dataset dimuat menggunakan pandas dan ukurannya ditampilkan.

---

## 🧪 2. Kriteria Seleksi Akhir

Dataset disaring berdasarkan kondisi berikut:

- **Rating (`averageRating`) tidak kosong** — artinya film sudah memiliki penilaian dari pengguna.
- **Genre (`genres`) tidak kosong** — genre diperlukan untuk filtering dan analisis.
- **Overview tidak mengandung teks "NOT_FOUND"** — hanya overview asli atau masih kosong (`NaN`) yang diperboleh

In [ ]:
import pandas as pd

# Path ke file enriched
file_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Filal_clean.csv"
output_cleaned = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Sempurna.csv"

# Load data
df = pd.read_csv(file_path)

# Tampilkan ukuran awal
print(f"📦 Dataset awal: {len(df)} baris")

# Tetapkan baris yang:
# - memiliki rating
# - memiliki genre
# - overview TIDAK berisi "NOT_FOUND"
# - biarkan overview kosong (NaN) jika belum diproses
df_clean = df[
    df["averageRating"].notna() &                # Rating tidak kosong
    df["genres"].notna() &                       # Genre tidak kosong
    ((df["overview"].isna()) | (df["overview"] != "NOT_FOUND"))  # Buang yang overview == "NOT_FOUND" saja
]

# Simpan hasil
df_clean.to_csv(output_cleaned, index=False)
print(f"✅ Dataset bersih disimpan: {output_cleaned}")
print(f"📉 Jumlah baris setelah dibersihkan: {len(df_clean)}")

📦 Dataset awal: 160994 baris
✅ Dataset bersih disimpan: /content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Sempurna.csv
📉 Jumlah baris setelah dibersihkan: 148614


In [ ]:
import pandas as pd

# Path ke file enriched
file_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Sempurna.csv"

# Load data
df = pd.read_csv(file_path)
total_rows = len(df)

# Baris yang belum terisi sama sekali (poster dan overview keduanya NaN)
unfilled_rows = df[df["poster_url"].isna() & df["overview"].isna()]
count_unfilled = len(unfilled_rows)
percent_unfilled = (count_unfilled / total_rows) * 100

# Sisanya dianggap terisi (sudah diproses), baik valid atau "NOT_FOUND"
count_filled = total_rows - count_unfilled
percent_filled = (count_filled / total_rows) * 100

# Tampilkan ringkasan
print("📊 Ringkasan Status Dataset (NOT_FOUND dianggap terisi):\n")
print(f"- Jumlah total baris        : {total_rows}")
print(f"- Jumlah baris sudah diproses : {count_filled} ({percent_filled:.2f}%) — memiliki minimal poster/sinopsis")
print(f"- Jumlah baris belum terisi  : {count_unfilled} ({percent_unfilled:.2f}%) — masih kosong (belum pernah diproses)")


# Hitung kondisi yang diminta
rating_missing = df["averageRating"].isna().sum()
genre_missing = df["genres"].isna().sum()
overview_nf = (df["overview"] == "NOT_FOUND").sum()
poster_nf = (df["poster_url"] == "NOT_FOUND").sum()
year_missing = df["startYear"].isna().sum()

# Tambahan: Baris yang BELUM dilakukan pengambilan data TMDb (masih kosong dan bukan "NOT_FOUND")
pending_tmdb = df[
    ((df["poster_url"].isna()) & (df["poster_url"] != "NOT_FOUND")) |
    ((df["overview"].isna()) & (df["overview"] != "NOT_FOUND"))
]
pending_tmdb_count = len(pending_tmdb)

# Tampilkan hasil
print("📊 Rekap Data Tidak Lengkap / Potensial Dibuang:")
print(f"- Baris tanpa rating              : {rating_missing}")
print(f"- Baris tanpa genre               : {genre_missing}")
print(f"- Baris overview 'NOT_FOUND'      : {overview_nf}")
print(f"- Baris poster 'NOT_FOUND'        : {poster_nf}")
print(f"- Baris tanpa tahun rilis         : {year_missing}")
print(f"- Baris BELUM diambil dari API TMDb: {pending_tmdb_count}")

📊 Ringkasan Status Dataset (NOT_FOUND dianggap terisi):

- Jumlah total baris        : 148614
- Jumlah baris sudah diproses : 148614 (100.00%) — memiliki minimal poster/sinopsis
- Jumlah baris belum terisi  : 0 (0.00%) — masih kosong (belum pernah diproses)
📊 Rekap Data Tidak Lengkap / Potensial Dibuang:
- Baris tanpa rating              : 0
- Baris tanpa genre               : 0
- Baris overview 'NOT_FOUND'      : 0
- Baris poster 'NOT_FOUND'        : 10672
- Baris tanpa tahun rilis         : 0
- Baris BELUM diambil dari API TMDb: 0


# 🧾 Konversi Dataset CSV ke Format Parquet

Pada tahap ini, dataset final yang telah dibersihkan (`imdb_tmdb_Sempurna.csv`) dikonversi ke format **Parquet**.

---

## 🔄 Alur Proses

1. **Membaca file CSV**: Dataset hasil akhir dimuat dari file CSV menggunakan `pandas.read_csv()`.
2. **Menyimpan ke Parquet**: Dataset kemudian disimpan ke format `.parquet` menggunakan `pandas.DataFrame.to_parquet()` dengan engine `pyarrow`.
3. **Output akhir**: File disimpan sebagai `imdb_tmdb_Sempurna.parquet`, yang lebih efisien untuk komputasi dan penyimpanan dibanding CSV.

---

## 🎯 Tujuan

- **Optimisasi penyimpanan**: File Parquet menggunakan kompresi kolom yang efisien.
- **Performa**: Akses data besar jadi lebih cepat, khususnya saat digunakan dalam pipeline ML atau query analitik seperti di **AWS Athena** atau **Pandas + Dask**.

In [ ]:
import pandas as pd

# Path ke file CSV bersih
csv_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Sempurna.csv"

# Path tujuan file Parquet
parquet_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Sempurna.parquet"

# Load CSV
df = pd.read_csv(csv_path)

# Simpan ke Parquet
df.to_parquet(parquet_path, engine='pyarrow', index=False)

print(f"✅ File berhasil dikonversi ke Parquet: {parquet_path}")

✅ File berhasil dikonversi ke Parquet: /content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Sempurna.parquet


In [ ]:
import pandas as pd

# Path ke file Parquet
file_path = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Sempurna.parquet"

# Load file Parquet
df = pd.read_parquet(file_path)

# Tampilkan informasi
print("📦 Jumlah baris dan kolom:")
print(df.shape)

print("\n📋 Daftar kolom:")
print(df.columns.tolist())

print("\n🔍 Contoh 5 baris pertama:")
print(df.head())

print("\n🧾 Struktur baris pertama (to_dict):")
print(df.head(1).to_dict())

📦 Jumlah baris dan kolom:
(148614, 13)

📋 Daftar kolom:
['tconst', 'primaryTitle', 'originalTitle', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'directors', 'writers', 'actors', 'poster_url', 'overview']

🔍 Contoh 5 baris pertama:
      tconst                                       primaryTitle  \
0  tt0035423                                     Kate & Leopold   
1  tt0062336  The Tango of the Widower and Its Distorting Mi...   
2  tt0069049                         The Other Side of the Wind   
3  tt0070596                                  Socialist Realism   
4  tt0082328                                 Embodiment of Evil   

                               originalTitle  startYear  runtimeMinutes  \
0                             Kate & Leopold     2001.0           118.0   
1  El tango del viudo y su espejo deformante     2020.0            70.0   
2                 The Other Side of the Wind     2018.0           122.0   
3                     El realismo social

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 🎬 Embedding Metadata Film dengan BERT Multilingual

Dokumen ini menjelaskan tahapan pembuatan embedding dari metadata film menggunakan model `paraphrase-multilingual-MiniLM-L12-v2` dari Sentence-Transformers. Hasil embedding akan disimpan dalam format `.pkl` dan digunakan dalam sistem rekomendasi berbasis vektor.

---

## 1. Menyiapkan Path Dataset dan Output

Kita mendefinisikan lokasi dataset `.parquet` hasil proses pembersihan, serta nama file output untuk menyimpan hasil embedding `.pkl`.

---

## 2. Memuat Dataset

Dataset dimuat menggunakan `pandas.read_parquet()` untuk efisiensi pemrosesan data besar. Dataset ini memuat informasi lengkap seperti judul, genre, aktor, sutradara, dan rating film.

---

## 3. Menampilkan Kolom Dataset

Bagian ini menampilkan semua kolom yang tersedia untuk memastikan semua atribut penting ada sebelum dibuat menjadi embedding.

---

## 4. Memilih Perangkat (CPU atau GPU)

Jika tersedia, proses akan menggunakan GPU (`cuda`) agar encoding dapat berjalan lebih cepat. Jika tidak ada GPU, maka akan menggunakan CPU.

---

## 5. Memuat Model Sentence-BERT Multilingual

Model `paraphrase-multilingual-MiniLM-L12-v2` digunakan karena mendukung berbagai bahasa dan ringan untuk digunakan di lingkungan terbatas seperti Google Colab atau lokal.

---

## 6. Menyiapkan Nilai Default dan Konversi Kolom

Semua kolom numerik dan string dikonversi menjadi string agar dapat digabung menjadi teks deskriptif. Nilai kosong akan diisi dengan default agar tidak mengganggu proses encoding.

---

## 7. Menggabungkan Metadata Film Menjadi Teks

Kolom seperti judul, genre, sutradara, dan aktor digabung menjadi satu string deskriptif untuk setiap film. Format ini akan digunakan untuk diubah menjadi embedding.

---

## 8. Proses Batch Embedding

Untuk efisiensi memori dan kecepatan, metadata diubah menjadi embedding dalam batch. Proses ini dilakukan bertahap dan ditampilkan waktu estimasi setiap batch.

---

## 9. Menyimpan Hasil Embedding

Hasil embedding vektor disimpan dalam file `.pkl` menggunakan `pickle`. File ini akan digunakan di tahap selanjutnya untuk sistem rekomendasi film berbasis similarity.

---

## ✅ Akhir Proses

Waktu total yang dibutuhkan ditampilkan untuk melihat performa proses embedding seluruh dataset film.

In [ ]:
import os
import pandas as pd
import pickle
import torch
import time
from sentence_transformers import SentenceTransformer

# === 📌 1. Path dataset dan output ===
DATASET_PATH = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/"
imdb_file = os.path.join(DATASET_PATH, "imdb_tmdb_Sempurna.parquet")
output_pkl = os.path.join(DATASET_PATH, "rich_movie_embeddings.pkl")

print(f"\U0001F4C2 Memuat dataset dari `{imdb_file}`...")
df_movies = pd.read_parquet(imdb_file)

# === 📌 2. Cek kolom yang tersedia ===
print("\U0001F50D Kolom yang tersedia dalam dataset:")
print(df_movies.columns.tolist())

# === 📌 3. Gunakan GPU jika tersedia ===
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"\U0001F680 Menggunakan perangkat: {device}")

# === 📌 4. Load model BERT multilingual ===
print("\U0001F504 Memuat model `paraphrase-multilingual-MiniLM-L12-v2`...")
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", device=device)

# === 📌 5. Isi kolom yang hilang dan konversi ke string ===
print("\U0001F504 Konversi kolom numerik ke string...")
fill_defaults = {
    "averageRating": 0,
    "numVotes": 0,
    "startYear": "Unknown",
    "runtimeMinutes": "Unknown",
    "isAdult": "0",
    "endYear": "Unknown",
    "titleType": "Movie",
    "directors": "",
    "writers": "",
    "actors": "",
    "genres": "",
    "originalTitle": ""
}

for col, default in fill_defaults.items():
    if col not in df_movies.columns:
        df_movies[col] = default
    df_movies[col] = df_movies[col].fillna(default).astype(str)

# === 📌 6. Gabungkan metadata untuk teks ===
print("\U0001F504 Menggabungkan informasi film untuk embedding...")
df_movies["metadata"] = (
    "ID: " + df_movies["tconst"] + " | " +
    df_movies["primaryTitle"] + " (" + df_movies["originalTitle"] + ") | " +
    "Jenis: " + df_movies["titleType"] + " | " +
    "Genre: " + df_movies["genres"] + " | " +
    "Sutradara: " + df_movies["directors"] + " | " +
    "Penulis: " + df_movies["writers"] + " | " +
    "Aktor: " + df_movies["actors"] + " | " +
    "Rating: " + df_movies["averageRating"] + " | " +
    "Votes: " + df_movies["numVotes"] + " | " +
    "Tahun Rilis: " + df_movies["startYear"] + " | " +
    "Tahun Akhir: " + df_movies["endYear"] + " | " +
    "Durasi: " + df_movies["runtimeMinutes"] + " min | " +
    "Dewasa: " + df_movies["isAdult"]
)

# === 📌 7. Embedding bertahap ===
batch_size = 4096
metadata_list = df_movies["metadata"].tolist()
total_batches = (len(metadata_list) + batch_size - 1) // batch_size
print(f"\U0001F504 Memproses {len(metadata_list)} metadata dalam {total_batches} batch (batch size: {batch_size})...")

metadata_embeddings = []
start_time = time.time()

for i in range(total_batches):
    start = i * batch_size
    end = min((i + 1) * batch_size, len(metadata_list))
    batch = metadata_list[start:end]

    batch_start_time = time.time()
    print(f"\U0001F504 Batch {i + 1}/{total_batches} ({start}–{end})...")

    with torch.no_grad():
        batch_embeddings = model.encode(batch, show_progress_bar=False, batch_size=128)

    metadata_embeddings.extend(batch_embeddings)

    elapsed = time.time() - batch_start_time
    remaining = total_batches - (i + 1)
    print(f"✅ Batch {i + 1} selesai dalam {elapsed:.2f} detik | Estimasi sisa: {elapsed * remaining / 60:.2f} menit")

# === 📌 8. Simpan hasil embedding ===
print(f"💾 Menyimpan embedding ke `{output_pkl}`...")
with open(output_pkl, "wb") as f:
    pickle.dump(metadata_embeddings, f)

total_time = (time.time() - start_time) / 60
print(f"✅ Selesai! Waktu total: {total_time:.2f} menit")


📂 Memuat dataset dari `/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/imdb_tmdb_Sempurna.parquet`...
🔍 Kolom yang tersedia dalam dataset:
['tconst', 'primaryTitle', 'originalTitle', 'startYear', 'runtimeMinutes', 'genres', 'averageRating', 'numVotes', 'directors', 'writers', 'actors', 'poster_url', 'overview']
🚀 Menggunakan perangkat: cuda
🔄 Memuat model `paraphrase-multilingual-MiniLM-L12-v2`...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

🔄 Konversi kolom numerik ke string...
🔄 Menggabungkan informasi film untuk embedding...
🔄 Memproses 148614 metadata dalam 37 batch (batch size: 4096)...
🔄 Batch 1/37 (0–4096)...
✅ Batch 1 selesai dalam 9.38 detik | Estimasi sisa: 5.63 menit
🔄 Batch 2/37 (4096–8192)...
✅ Batch 2 selesai dalam 8.46 detik | Estimasi sisa: 4.94 menit
🔄 Batch 3/37 (8192–12288)...
✅ Batch 3 selesai dalam 8.71 detik | Estimasi sisa: 4.93 menit
🔄 Batch 4/37 (12288–16384)...
✅ Batch 4 selesai dalam 8.84 detik | Estimasi sisa: 4.86 menit
🔄 Batch 5/37 (16384–20480)...
✅ Batch 5 selesai dalam 8.78 detik | Estimasi sisa: 4.68 menit
🔄 Batch 6/37 (20480–24576)...
✅ Batch 6 selesai dalam 9.08 detik | Estimasi sisa: 4.69 menit
🔄 Batch 7/37 (24576–28672)...
✅ Batch 7 selesai dalam 9.46 detik | Estimasi sisa: 4.73 menit
🔄 Batch 8/37 (28672–32768)...
✅ Batch 8 selesai dalam 8.80 detik | Estimasi sisa: 4.25 menit
🔄 Batch 9/37 (32768–36864)...
✅ Batch 9 selesai dalam 9.03 detik | Estimasi sisa: 4.21 menit
🔄 Batch 10/37 (368

In [2]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 71.3 MB/s eta 0:00:00


In [6]:
import pandas as pd
import numpy as np
import faiss
import pickle
from sentence_transformers import SentenceTransformer

# 🔧 Path
DATASET_PATH = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/"
pkl_path = DATASET_PATH + "rich_movie_embeddings.pkl"
parquet_path = DATASET_PATH + "imdb_tmdb_Sempurna.parquet"

# 📥 Load data & embedding
df = pd.read_parquet(parquet_path)
with open(pkl_path, "rb") as f:
    embeddings = np.array(pickle.load(f))

# 🔍 Normalisasi & Index
faiss.normalize_L2(embeddings)
index = faiss.IndexFlatIP(embeddings.shape[1])
index.add(embeddings)

# 🧠 Load model BERT
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# 💡 Coba query
query = "film tentang perang luar angkasa"
query_vec = model.encode([query])
faiss.normalize_L2(query_vec)

# 🔍 Cari 5 film teratas
D, I = index.search(query_vec, 10)

# 📋 Tampilkan hasil
print("\n🎬 Rekomendasi Berdasarkan Query:")
for rank, i in enumerate(I[0]):
    print(f"{rank+1}. {df.iloc[i]['primaryTitle']} ({df.iloc[i]['startYear']})")
    print(f"   🎭 Genre : {df.iloc[i]['genres']}")
    print(f"   ⭐ Rating: {df.iloc[i]['averageRating']}")
    print(f"   📖 Sinopsis: {df.iloc[i]['overview'][:200]}...\n")


🎬 Rekomendasi Berdasarkan Query:
1. The Space Invaders: In Search of Lost Time (2012.0)
   🎭 Genre : Biography,Documentary,History
   ⭐ Rating: 6.5
   📖 Sinopsis: Beginning with Space Invaders in 1978, arcade games began to appear everywhere. By 1982, there were 13,000 dedicated arcade locations across North America. It was the Golden Age of Arcade Games, gener...

2. Spaceman (2024.0)
   🎭 Genre : Adventure,Drama,Sci-Fi
   ⭐ Rating: 5.7
   📖 Sinopsis: Six months into a solo mission, a lonely astronaut confronts the cracks in his marriage with help from a mysterious creature he discovers on his ship....

3. 2001: A Space Travesty (2000.0)
   🎭 Genre : Comedy,Sci-Fi
   ⭐ Rating: 3.5
   📖 Sinopsis: When odd reports are received through official channels stating that the President of the United States is being held captive on a secret international moon base called Vegan and that he has been repl...

4. Interstellar Civil War: Shadows of the Empire (2017.0)
   🎭 Genre : Action,Adventure,

In [ ]:
from huggingface_hub import snapshot_download
import os

# Tentukan path tujuan penyimpanan
target_dir = "/content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/multilingual_bert/"

# Buat folder jika belum ada
os.makedirs(target_dir, exist_ok=True)

# 📥 Download model dan simpan ke folder target
print("📥 Mendownload model ke:", target_dir)
model_path = snapshot_download(
    repo_id="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    local_dir=target_dir,
    local_dir_use_symlinks=False
)
print("✅ Model berhasil diunduh ke:", model_path)

📥 Mendownload model ke: /content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/multilingual_bert/


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:933: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(


Fetching 28 files:   0%|          | 0/28 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


.gitattributes:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tf_model.h5:   0%|          | 0.00/471M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


unigram.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

✅ Model berhasil diunduh ke: /content/drive/Othercomputers/My MacBook Pro/S2KecerdasanBuatan/2. Semester 2/MMAI6002 Pengembangan Perangkat Lunak dan Manajemen Proyek/4_Final/Versi 2/imdb/multilingual_bert
